In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---- Simulated versions of your src functions ----
def generate_synthetic_prices(n_assets=4, n_days=1250, seed=None):
    if seed is not None:
        np.random.seed(seed)
    prices = np.cumprod(1 + np.random.normal(0.0005, 0.01, size=(n_days, n_assets)), axis=0) * 100
    return pd.DataFrame(prices, columns=[f"Asset_{i+1}" for i in range(n_assets)])

def example_weights(n_assets):
    w = np.ones(n_assets) / n_assets
    return w

def portfolio_returns_from_prices(prices, weights):
    returns = prices.pct_change().dropna()
    portfolio_ret = returns.dot(weights)
    return portfolio_ret

def parametric_var(returns, weights=None, alpha=0.05):
    mean = returns.mean()
    std = returns.std()
    from scipy.stats import norm
    return - (mean + std * norm.ppf(alpha))

def historical_var(returns, alpha=0.05):
    return - np.percentile(returns, 100 * alpha)

def monte_carlo_var(mu, cov, weights, n_sims=50000, alpha=0.05):
    sims = np.random.multivariate_normal(mu, cov, size=n_sims)
    port_sims = sims.dot(weights)
    return - np.percentile(port_sims, 100 * alpha)

def test_portfolio_returns_shape():
    prices = generate_synthetic_prices(n_assets=3, n_days=50, seed=1)
    w = example_weights(3)
    prs = portfolio_returns_from_prices(prices, w)
    assert len(prs) == 49

def test_parametric_vs_historical_small():
    prices = generate_synthetic_prices(n_assets=1, n_days=500, seed=2)
    w = example_weights(1)
    prs = portfolio_returns_from_prices(prices, w)
    pvar = parametric_var(prs, weights=None, alpha=0.05)
    hvar = historical_var(prs, alpha=0.05)
    assert pvar >= 0 and hvar >= 0

# Run tests
test_portfolio_returns_shape()
test_parametric_vs_historical_small()
print("✅ All tests passed!")

✅ All tests passed!
